<a href="https://colab.research.google.com/github/anjana-psvel/Precog_Task/blob/main/pretrained_transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
df = pd.read_csv('/content/phrase_sim.csv')
df

,entity1,entity2
0,one data,a particular statistic
1,involved people,participating individuals
2,wide majority,broad bulk
3,robust selection,powerful choice
4,general point,basic fact
...,...,...
4997,large strain,big stress
4998,smallest mark,merest price raise
4999,huge load,massive shipment
5000,perhaps one,possibly a singular one


In [8]:
!pip install transformers -U

In [3]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 28.7 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3


In [4]:
from datasets import load_dataset

# Load a text dataset example
dataset = load_dataset("PiC/phrase_similarity")

# Accessing the dataset
train_data = dataset["train"]
test_data = dataset["test"]

# Print the first few examples
print(train_data[0])
print(test_data[0])


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/7004 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2000 [00:00<?, ? examples/s]

{'phrase1': 'newly formed camp', 'phrase2': 'recently made encampment', 'sentence1': 'newly formed camp is released from the membrane and diffuses across the intracellular space where it serves to activate pka.', 'sentence2': 'recently made encampment is released from the membrane and diffuses across the intracellular space where it serves to activate pka.', 'label': 0, 'idx': 0}
{'phrase1': 'air position', 'phrase2': 'posture while jumping', 'sentence1': 'In 1990, Petit accepted a full-time overnight on air position at gospel radio station WYLD-AM.', 'sentence2': 'In 1990, Petit accepted a full-time overnight on posture while jumping at gospel radio station WYLD-AM.', 'label': 0, 'idx': 0}


In [5]:
# prompt: create a dataframe with phrase1, phrase2, sentene1 and sentence2 with their label

import pandas as pd
data = {
    "phrase1": train_data["phrase1"],
    "phrase2": train_data["phrase2"],
    "sentence1": train_data["sentence1"],
    "sentence2": train_data["sentence2"],
    "label": train_data["label"]
}

df = pd.DataFrame(data)
df.head()


,phrase1,phrase2,sentence1,sentence2,label
0,newly formed camp,recently made encampment,newly formed camp is released from the membran...,recently made encampment is released from the ...,0
1,one data,a particular statistic,"According to one data, in 1910, on others – in...","According to a particular statistic, in 1910, ...",1
2,particular structure,specific edifice,Note that Fact 1 does not assume any particula...,Note that Fact 1 does not assume any specific ...,0
3,involved people,participating individuals,Assessment-Center are usually group-processes ...,Assessment-Center are usually group-processes ...,1
4,different cross,opposing inquiries,"At the end of the 1980s, a different cross had...","At the end of the 1980s, a opposing inquiries ...",0


In [6]:
# prompt: how many records are there in this df?

print(df.shape[0])


7004


In [7]:
# prompt: take only first 2000 records

df = df.iloc[:2000]


In [9]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
import torch
from transformers import TrainingArguments, Trainer
from transformers import BertTokenizer, BertForSequenceClassification

In [10]:
from transformers import BertTokenizer, BertForSequenceClassification
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased',num_labels=2)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
